In [88]:
import requests # pour faire des requêtes
import lxml.html as lh # pour l'analyse des tags
import pandas as pd    # pour structurer les données
import csv
 
url = 'https://auvergnerhonealpes.simplon.co/candidatures.html'
# récupérer l'adresse de simplon pour les formations

uri_pages = '?page=' # pour changer les pages
nbPages = 400        #400 pages possibles

In [89]:
#différents tags pour récupérer les différentes catégories d'informations nécessaires
tags = ['//h2[@class="card-title"]', \
        '//div[@class="card-content"]', \
        '//span[@class="day"]', \
        '//span[@class="month"]', \
        '//div[@class="year"]', \
        '//div[@class="card-session-info calendar"]', \
        '//div[@class="card-session-infos"]', \
       ]

In [90]:
#nom colonnes  pour information formation
cols = ['Nom', 'Lieu', 'Jour', 'Mois', 'Annee', 'Debut', 'Information']

In [91]:
#requete pour recuperer l'information de l'url a renseigné
page = requests.get(url)

In [92]:
#lire le contenu de la requete
doc = lh.fromstring(page.content)

In [93]:
doc.xpath(tags[1])[0].text_content()

'\n            \n                            \n            Simplon Lyon        '

In [94]:
tags

['//h2[@class="card-title"]',
 '//div[@class="card-content"]',
 '//span[@class="day"]',
 '//span[@class="month"]',
 '//div[@class="year"]',
 '//div[@class="card-session-info calendar"]',
 '//div[@class="card-session-infos"]']

card-title = intitulé de la formation
card-content= le lieu et comment la formation est pris en charge
day = le jour  ou aura lieu cette formation
month = le mois ou aura lieu cette formation
card-session-infos = le mois ainsi que l'année de début
card-session-info calendar = le mois ainsi que l'année de début, la durée de la formation, le lieu

In [95]:
for i in doc.xpath(tags[6]):
    print(i.text_content())
    #print(i.text_content().strip())


            
                
                    event
                

                Début : novembre 2021
            
            
                
                    hourglass_empty
                
                                    inférieur à 3 mois
                            
            
                
                    location_on
                
                                                            Auvergne-Rhône-Alpes
                                                

            
        

            
                
                    event
                

                Début : décembre 2021
            
            
                
                    hourglass_empty
                
                                    au-delà de 7 mois
                            
            
                
                    location_on
                
                                                            Auvergne-Rhône-Alpes
                       

In [101]:
content = []
for i in range(len(tags)):# récupérer info pour les 7 tags sous format xpath
        content.append(doc.xpath(tags[i]))
        
for j in range(len(tags)):# pour tous les tags (7)
        print(j)
        test = pd.DataFrame([content[j][i].text_content().strip() for i in range(len(content[i]))], columns=[cols[j]])

0
1
2
3
4
5
6


In [105]:
tags[6]

'//div[@class="card-session-infos"]'

In [103]:
j

6

In [102]:
pd.DataFrame([content[j][i].text_content().strip() for i in range(len(content[i]))], columns=[cols[j]])

,Information
0,event\n \n\n Déb...
1,event\n \n\n Déb...
2,event\n \n\n Déb...
3,event\n \n\n Déb...
4,event\n \n\n Déb...
5,event\n \n\n Déb...
6,event\n \n\n Déb...
7,event\n \n\n Déb...
8,event\n \n\n Déb...
9,event\n \n\n Déb...


In [83]:
def getPage(url): #fonction pour récupérer l'information de chaque page
    page = requests.get(url) # faire la requete
    doc = lh.fromstring(page.content) # avoir le contenu de la requete
 
    # Récupérer l'information pour chaque tag
    content = []
    for i in range(len(tags)):# récupérer info pour les 7 tags
        content.append(doc.xpath(tags[i])) # pour chaque tag récupère le contenu et mets dans la liste
 
    # Récupérer les données et les mettre dans un dataframe
    df_liste = []  #récupère dataframe
    for j in range(len(tags)):# pour tous les tags (7)
        tmp = pd.DataFrame([content[j][i].text_content().strip() for i in range(len(content[i]))], columns=[cols[j]])
        #tmp strip supprimer les espaces à droite et à gauche, supprimer les espaces à droute et à gauche de chaque mot. Et les mettre dans chaque colonne
        tmp['key'] = tmp.index 
        df_liste.append(tmp) #liste des df récupérer selon 
 
    # Crée un dataframe avec une colonne par tag(xpath)
    liste = df_liste[0] 
    for j in range(len(tags)-1): 
        liste = liste.join(df_liste[j+1], on='key', how='left', lsuffix='_l', rsuffix='_r')
        liste['key'] = liste.index
        del liste['key_l']
        del liste['key_r']
     
    return liste

In [84]:
def getPages(_nbPages, _url): 
    liste_finale = pd.DataFrame()
    for i in range (_nbPages): # reccueillir les informations de chaque dataframe
        liste = getPage(_url + uri_pages + str(i+1)) #stockage intermédiaire pour chaque page
        liste_finale = pd.concat([liste_finale, liste], ignore_index=True) # concaténer les résultats
    return liste_finale
 
liste_totale = getPages(nbPages, url)

In [2]:
liste_totale

,Nom,Lieu,Jour,Mois,Annee,Debut,Information,key
0,Apple Foundation Program - Développement mobil...,Simplon Lyon,14,11,2021,event\n \n\n Déb...,event\n \n\n Déb...,0
1,Apple Foundation Program - Développement mobil...,Simplon Lyon,03,10,2021,event\n \n\n Déb...,event\n \n\n Déb...,1
2,Ecole IA Microsoft By Simplon - Développeur.eu...,Simplon Grenoble,15,11,2021,event\n \n\n Déb...,event\n \n\n Déb...,2
3,Concepteur développeur d'applications - 100% D...,Alternance\n \n ...,21,10,2021,event\n \n\n Déb...,event\n \n\n Déb...,3
4,Administrateur·trice d’infrastructures sécuris...,Alternance\n \n ...,14,10,2021,event\n \n\n Déb...,event\n \n\n Déb...,4
...,...,...,...,...,...,...,...,...
4795,CléA Numérique - Compétences Digitales pour l’...,CPF\n \n ...,05,11,2021,event\n \n\n Déb...,event\n \n\n Déb...,7
4796,Gérer un projet en mobilisant les méthodes agi...,CPF\n \n ...,29,10,2021,event\n \n\n Déb...,event\n \n\n Déb...,8
4797,Développer avec JAVA EE,Simplon Lyon,31,10,2021,event\n \n\n Déb...,event\n \n\n Déb...,9
4798,CléA Numérique - Compétences Digitales pour l’...,CPF\n \n ...,27,12,2021,event\n \n\n Déb...,event\n \n\n Déb...,10


### Nettoyage des chaînes de caractères récupérées, enlever les caractères inutiles poure les colonnes polluées

In [4]:
liste_totale['Debut'] = liste_totale['Debut'].replace({'event':''}, regex= True)

In [5]:
liste_totale['Debut'] = liste_totale['Debut'].replace({'\n':''}, regex= True)
liste_totale['Debut'] = liste_totale['Debut'].replace({'Début':''}, regex= True)
liste_totale['Debut'] = liste_totale['Debut'].replace({':':''}, regex= True)

In [6]:
liste_totale

,Nom,Lieu,Jour,Mois,Annee,Debut,Information,key
0,Apple Foundation Program - Développement mobil...,Simplon Lyon,14,11,2021,novembre 2021,event\n \n\n Déb...,0
1,Apple Foundation Program - Développement mobil...,Simplon Lyon,03,10,2021,octobre 2021,event\n \n\n Déb...,1
2,Ecole IA Microsoft By Simplon - Développeur.eu...,Simplon Grenoble,15,11,2021,décembre 2021,event\n \n\n Déb...,2
3,Concepteur développeur d'applications - 100% D...,Alternance\n \n ...,21,10,2021,septembre 2021,event\n \n\n Déb...,3
4,Administrateur·trice d’infrastructures sécuris...,Alternance\n \n ...,14,10,2021,septembre 2021,event\n \n\n Déb...,4
...,...,...,...,...,...,...,...,...
4795,CléA Numérique - Compétences Digitales pour l’...,CPF\n \n ...,05,11,2021,novembre 2021,event\n \n\n Déb...,7
4796,Gérer un projet en mobilisant les méthodes agi...,CPF\n \n ...,29,10,2021,novembre 2021,event\n \n\n Déb...,8
4797,Développer avec JAVA EE,Simplon Lyon,31,10,2021,novembre 2021,event\n \n\n Déb...,9
4798,CléA Numérique - Compétences Digitales pour l’...,CPF\n \n ...,27,12,2021,décembre 2021,event\n \n\n Déb...,10


In [7]:
liste_totale['Information'] = liste_totale['Information'].replace({'event':''}, regex= True)
liste_totale['Information'] = liste_totale['Information'].replace({'\n':''}, regex= True)
liste_totale['Information'] = liste_totale['Information'].replace({'Début':''}, regex= True)
liste_totale['Information'] = liste_totale['Information'].replace({':':''}, regex= True)

In [8]:
liste_totale

,Nom,Lieu,Jour,Mois,Annee,Debut,Information,key
0,Apple Foundation Program - Développement mobil...,Simplon Lyon,14,11,2021,novembre 2021,novembre 202...,0
1,Apple Foundation Program - Développement mobil...,Simplon Lyon,03,10,2021,octobre 2021,octobre 2021...,1
2,Ecole IA Microsoft By Simplon - Développeur.eu...,Simplon Grenoble,15,11,2021,décembre 2021,décembre 202...,2
3,Concepteur développeur d'applications - 100% D...,Alternance\n \n ...,21,10,2021,septembre 2021,septembre 20...,3
4,Administrateur·trice d’infrastructures sécuris...,Alternance\n \n ...,14,10,2021,septembre 2021,septembre 20...,4
...,...,...,...,...,...,...,...,...
4795,CléA Numérique - Compétences Digitales pour l’...,CPF\n \n ...,05,11,2021,novembre 2021,novembre 202...,7
4796,Gérer un projet en mobilisant les méthodes agi...,CPF\n \n ...,29,10,2021,novembre 2021,novembre 202...,8
4797,Développer avec JAVA EE,Simplon Lyon,31,10,2021,novembre 2021,novembre 202...,9
4798,CléA Numérique - Compétences Digitales pour l’...,CPF\n \n ...,27,12,2021,décembre 2021,décembre 202...,10


Récupération des éléments pour indiquer la durée 

In [9]:
df = liste_totale.copy()
df["duree1"] = df["Information"].str.split().str[3]
df["duree2"] = df["Information"].str.split().str[4]
df["duree3"] = df["Information"].str.split().str[5]
df["duree4"] = df["Information"].str.split().str[6]

Récupérer la région  de la formation

In [10]:
df["Localisation"] = df["Information"].str.split().str[8]

In [11]:
df = df.drop('Information', axis=1)

Regrouper les informations de la durée (ex inférieur à un mois)

In [12]:
df['Duree'] = df['duree1'] + ' ' + df['duree2'] + ' ' + df['duree3'] + ' ' + df['duree4']

In [13]:
df = df.drop(['duree1', 'duree2', 'duree3', 'duree4',], axis=1)

Assemblage de la date limite

In [14]:
df['datelimite'] = df['Jour'] + '-' + df['Mois'] + '-' + df['Annee']

Suppression des colonnes que nous n'allons pas utiliser

In [15]:
df = df.drop(['Jour', 'Mois', 'Annee'], axis=1)

In [16]:
pd.set_option('display.max_rows', 56)

In [17]:
df = df.head(57)

In [18]:
df

,Nom,Lieu,Debut,key,Localisation,Duree,datelimite
0,Apple Foundation Program - Développement mobil...,Simplon Lyon,novembre 2021,0,Auvergne-Rhône-Alpes,inférieur à 3 mois,14-11-2021
1,Apple Foundation Program - Développement mobil...,Simplon Lyon,octobre 2021,1,Auvergne-Rhône-Alpes,inférieur à 3 mois,03-10-2021
2,Ecole IA Microsoft By Simplon - Développeur.eu...,Simplon Grenoble,décembre 2021,2,Auvergne-Rhône-Alpes,au-delà de 7 mois,15-11-2021
3,Concepteur développeur d'applications - 100% D...,Alternance\n \n ...,septembre 2021,3,Auvergne-Rhône-Alpes,au-delà de 7 mois,21-10-2021
4,Administrateur·trice d’infrastructures sécuris...,Alternance\n \n ...,septembre 2021,4,Auvergne-Rhône-Alpes,au-delà de 7 mois,14-10-2021
...,...,...,...,...,...,...,...
52,Administrateur·trice d’infrastructures sécuris...,Alternance\n \n ...,septembre 2021,4,Auvergne-Rhône-Alpes,au-delà de 7 mois,14-10-2021
53,Refugeeks,Simplon Lyon,novembre 2021,5,Auvergne-Rhône-Alpes,inférieur à 3 mois,21-10-2021
54,Gérer un projet en mobilisant les méthodes agi...,CPF\n \n ...,novembre 2021,6,Auvergne-Rhône-Alpes,inférieur à 10 jours,05-11-2021
55,CléA Numérique - Compétences Digitales pour l’...,CPF\n \n ...,novembre 2021,7,Auvergne-Rhône-Alpes,inférieur à 10 jours,05-11-2021


Exportation en csv

In [ ]:
df.to_csv('formation_simplon.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)